## 1a. Import the libraries and sample data

Credits: [Selva Prabhakaran](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)

Download [`newsgroups.json` here provided by @selva86](https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json)

Ensure that spacy `en_core_web_sm` is installed. You can do so by invoking `python -m spacy download en_core_web_sm` in the terminal.

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

# Set stopwords
sw = stopwords.words('english')
sw.extend(['from', 'subject', 're', 'edu', 'use'])

# Import Dataset
df = pd.read_json('newsgroups.json')
print(df.target_names.unique())
df.head()

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


## 1b. Clean-up sentences and Tokenize words

In [7]:
# Remove emails and newline characters
data = df.content.values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

# Tokenize words and clean-up text - deacc=True removes punctuations
data_words = [gensim.utils.simple_preprocess(str(s), deacc=True) for s in sent_to_words(data)]

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:3: DeprecationWarning: invalid escape sequence \S
<>:4: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \S
<>:4: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \S
<>:4: DeprecationWarning: invalid escape sequence \s
<ipython-input-7-150a27fb396d>:3: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-7-150a27fb396d>:4: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


## 1c. Remove Stopwords, make Bigrams and Lemmatize

In [11]:
# Remove Stopwords
data_words_nostops = [[word for word in simple_preprocess(str(doc)) if word not in sw] for doc in data_words]

# Form Bigrams - Higher threshold fewer phrases.
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
bigram_mod = gensim.models.phrases.Phraser(bigram)

data_words_bigrams = [bigram_mod[doc] for doc in data_words_nostops]

# Use tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization and keep only noun, adj, vb, adv
allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']
data_lemmatized = [[token.lemma_ for token in nlp(' '.join(s)) if token.pos_ in allowed_postags] for s in data_words_bigrams]

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 1d. Create the Dictionary and Corpus

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 2. Build the LDA model

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 3a. Display keywords in topics

To get a sense, for each topic,

<img src="https://www.machinelearningplus.com/wp-content/uploads/2018/03/Inferring-Topic-from-Keywords-1200x781.png?ezimgfmt=ng:webp/ngcb4" width="600" height="auto" />

(Source: [Selva Prabhakaran, 2018](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/))

In [76]:
from IPython.display import display, HTML
import pandas as pd

top_topics_rank = {t[0]: r + 1 for r, t in enumerate(lda_model.show_topics())}
topics_df = pd.DataFrame([
    [no, top_topics_rank.get(no, '-'), kws.split(' + ')] for no, kws in lda_model.show_topics(num_topics=20)
], columns=['Topic No.', 'Rank', 'Top 10 Keywords (a + b)'])

display(HTML(topics_df.to_html(index=False)))

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic No.,Rank,Top 10 Keywords (a + b)
0,1,"[0.065*""data"", 0.034*""enable"", 0.011*""microsoft"", 0.003*""efficiently"", 0.000*""textual"", 0.000*""slave"", 0.000*""jumper"", 0.000*""cp_ut"", 0.000*""master_slave"", 0.000*""latch""]"
1,4,"[0.065*""scsi"", 0.064*""mb"", 0.058*""ide"", 0.054*""headache"", 0.044*""gateway"", 0.029*""water"", 0.028*""oil"", 0.025*""nuclear"", 0.023*""heat"", 0.022*""cylinder""]"
2,-,"[0.046*""gun"", 0.029*""whole"", 0.024*""bike"", 0.020*""black"", 0.019*""draw"", 0.019*""carry"", 0.017*""white"", 0.015*""police"", 0.015*""ride"", 0.014*""safety""]"
3,-,"[0.055*""year"", 0.049*""team"", 0.048*""game"", 0.035*""play"", 0.033*""win"", 0.016*""season"", 0.015*""fan"", 0.015*""last"", 0.015*""hit"", 0.014*""first""]"
4,-,"[0.053*""reality"", 0.049*""distribution_na"", 0.036*""concept"", 0.033*""boy"", 0.030*""poor"", 0.030*""parent"", 0.029*""door"", 0.028*""assumption"", 0.025*""benefit"", 0.022*""blood""]"
5,6,"[0.039*""number"", 0.032*""list"", 0.023*""include"", 0.023*""copy"", 0.023*""player"", 0.019*""name"", 0.016*""return"", 0.015*""section"", 0.015*""appreciate"", 0.015*""tape""]"
6,-,"[0.052*""kill"", 0.039*""soldier"", 0.035*""village"", 0.024*""turk"", 0.023*""turkish"", 0.023*""listen"", 0.022*""murder"", 0.021*""terrorism"", 0.020*""armenian"", 0.015*""girl""]"
7,-,"[0.096*""suggest"", 0.066*""community"", 0.048*""united_state"", 0.034*""impact"", 0.030*""weight"", 0.023*""legitimate"", 0.022*""role"", 0.022*""consideration"", 0.021*""approve"", 0.020*""progress""]"
8,9,"[0.035*""people"", 0.027*""say"", 0.018*""believe"", 0.016*""evidence"", 0.015*""reason"", 0.011*""fact"", 0.011*""claim"", 0.010*""think"", 0.010*""mean"", 0.009*""law""]"
9,-,"[0.133*""space"", 0.041*""launch"", 0.035*""earth"", 0.034*""mission"", 0.033*""corporation"", 0.033*""orbit"", 0.030*""satellite"", 0.027*""moon"", 0.023*""flight"", 0.017*""fuel""]"


## 3b. Compute Perplexity and Coherence scores

Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is

In [77]:
# Compute Perplexity - a measure of how good the model is. lower the better.
print('Perplexity: ', lda_model.log_perplexity(corpus)) 

# Compute Coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: ', coherence_lda)

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  -13.974043489993846 


Coherence Score:  0.4619199087319347


## 3d. Visualize the topics-keywords 

In [80]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.354994 -0.129912       1        1  38.458818
8     -0.327101 -0.182744       2        1  15.112065
19    -0.280116  0.024674       3        1  11.395874
12    -0.271249  0.193610       4        1  11.248502
5     -0.111614  0.090560       5        1   4.271919
14    -0.114924  0.300137       6        1   3.932927
3     -0.102887 -0.118382       7        1   3.821553
2     -0.020411  0.015088       8        1   3.037166
10     0.037290  0.068946       9        1   1.548845
15     0.045296 -0.089874      10        1   1.465651
6      0.039440 -0.144615      11        1   1.353174
4      0.143289 -0.007973      12        1   0.781976
9      0.123659  0.024262      13        1   0.776847
17     0.128722 -0.010772      14        1   0.666927
7      0.167247 -0.013408      15        1   0.502733
18     0.167925 -0.006115      16        1   0.498334
1      0.167095 -0.014461      17        1   0.454553
11     0.156967  0.032488      18        1   0.427699
13     0.195736 -0.014656      19        1   0.180424
0      0.210629 -0.016852      20        1   0.064012, topic_info=         Term          Freq         Total Category  logprob  loglift
148    people   7554.000000   7554.000000  Default  30.0000  30.0000
41       year   5086.000000   5086.000000  Default  29.0000  29.0000
18       line  17949.000000  17949.000000  Default  28.0000  28.0000
212     write  14101.000000  14101.000000  Default  27.0000  27.0000
377     drive   2758.000000   2758.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
4761       sp      0.013831      1.157945  Topic20 -10.9843   2.9263
4894   select      0.013831      1.157964  Topic20 -10.9843   2.9263
5343  install      0.013831      1.157967  Topic20 -10.9843   2.9263
4689   theory      0.013830      1.157953  Topic20 -10.9843   2.9263
3941  connect      0.013830      1.157989  Topic20 -10.9843   2.9263

[866 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
972       3  0.999533       _
366      17  0.969686  absurd
1378      2  0.973441  accept
1378      3  0.026178  accept
87        4  0.914570  access
...     ...       ...     ...
426      15  0.986932  writer
41        1  0.229446    year
41        3  0.248517    year
41        7  0.521807    year
2262     16  0.991774    zone

[1071 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 9, 20, 13, 6, 15, 4, 3, 11, 16, 7, 5, 10, 18, 8, 19, 2, 12, 14, 1])